# Using LogReg for feature selection

In [ ]:
import lamindb as ln
import seaborn as sns
sns.set_theme()
%config InlineBackend.figure_formats = ['svg']
%load_ext autoreload
%autoreload 2

In [ ]:
ln.track()

In [ ]:
artifact = ln.Artifact.using("laminlabs/arrayloader-benchmarks").get("D21D2K8697CY8tHE0001")

In [ ]:
adata = artifact.load()
adata

In [ ]:
adata.obs["cell_line"].nunique()

In [ ]:
adata.obs.value_counts("cell_line")

In [ ]:
import modlyn as mn
import lightning as L

logreg = mn.models.SimpleLogReg(
    adata=adata,
    label_column="cell_line",    
    learning_rate=1e-1,
)
logreg.fit(
    adata_train=adata,
    adata_val=adata[:100],
    train_dataloader_kwargs={
        "batch_size": 8,
    },
    max_epochs=4,
)

In [ ]:
logreg.plot_losses()

In [ ]:
logreg.plot_classification_report(adata)

In [ ]:
# ln.finish()